# Training

In [4]:
# wandb MA-sae-train: 03ca085d27243e9a7876d511e0c7402d861df34c

In [1]:
from sae_lens import SAETrainingRunner, LanguageModelSAERunnerConfig

In [2]:
# —— Schedule & batch sizing —————————————————————————————————————
total_steps            = 75_000                                    # ∼75 K gradient updates
batch_size_tokens      = 4_096                                     # 4 K tokens per step to keep an A100 busy
total_training_tokens  = total_steps * batch_size_tokens          # ≈300 M tokens

In [3]:
# —— Learning‐rate & sparsity warmups —————————————————————————————
lr_warmup_steps  = int(0.1 * total_steps)                          # 10% warmup avoids dead features
lr_decay_steps   = int(0.2 * total_steps)                          # 20% decay to stabilize later
l1_warmup_steps  = int(0.05 * total_steps)                         # 5% warmup on the L1 penalty

In [10]:
# —— SAE‐Lens configuration ———————————————————————————————————————
cfg = LanguageModelSAERunnerConfig(
    # — Data & model hooks —
    model_name                    = "EleutherAI/pythia-410m-deduped",
    hook_name                     = "blocks.4.hook_mlp_out",  # intervene on layer 4 MLP output
    hook_layer                    = 4,
    d_in                          = 1024,                    # MLP hidden‐size for Pythia-410m
    # dataset_path                  = "EleutherAI/pile",       # large, diverse language corpus
    dataset_path                  = "karpathy/tiny_shakespeare", 
    is_dataset_tokenized          = False,
    streaming                     = True,                     # stream directly from HF

    # — SAE architecture & sparsity —
    architecture                  = "standard",               # L1‐penalized SAE
    expansion_factor              = 16,                       # 16× overcomplete basis (≈32 K latents)
    l1_coefficient                = 5.0,                      # medium‐strength sparsity
    l1_warm_up_steps              = l1_warmup_steps,
    mse_loss_normalization        = None,                     # raw MSE

    # — Decoder & encoder initialization —
    b_dec_init_method             = "zeros",                  # start decoder at zero
    init_encoder_as_decoder_transpose = True,                # symmetry speeds convergence
    decoder_heuristic_init        = False,                     # geometric‐median bias helps stability

    # — Optimization & scheduling —
    lr                            = 5e-5,                     # standard Adam LR
    adam_beta1                    = 0.9,
    adam_beta2                    = 0.999,
    lr_scheduler_name             = "constant",
    lr_warm_up_steps              = lr_warmup_steps,
    lr_decay_steps                = lr_decay_steps,

    # — Context & batch sizing —
    train_batch_size_tokens       = batch_size_tokens,
    context_size                  = 256,                      # 256-token windows

    # — Reporting & logging —
    training_tokens               = total_training_tokens,
    feature_sampling_window       = 1_000,                    # report every 1 K steps
    log_to_wandb                  = True,
    wandb_project                 = "MA-sae-train",
    wandb_log_frequency           = 50,

    # — Miscellaneous —
    device                        = "cuda:0",
    seed                          = 42,
    n_checkpoints                 = 0,
    dtype                         = "float32"
)

In [11]:
# — Run!
sparse_autoencoder = SAETrainingRunner(cfg).run()

Loaded pretrained model EleutherAI/pythia-410m-deduped into HookedTransformer


/home/tilman.kerl/miniconda3/envs/refusal/lib/python3.11/site-packages/sae_lens/training/activations_store.py:301: UserWarning: Dataset is not tokenized. Pre-tokenizing will improve performance and allows for more control over special tokens. See https://jbloomaus.github.io/SAELens/training_saes/#pretokenizing-datasets for more info.
  warnings.warn(


Training SAE:   0%|          | 0/307200000 [00:00<?, ?it/s]


Refilling buffer:   0%|          | 0/10 [00:00<?, ?it/s]


Refilling buffer:  10%|█         | 1/10 [00:00<00:07,  1.13it/s]


Refilling buffer:  20%|██        | 2/10 [00:00<00:03,  2.34it/s]


Refilling buffer:  30%|███       | 3/10 [00:01<00:01,  3.57it/s]


Refilling buffer:  40%|████      | 4/10 [00:01<00:01,  4.78it/s]


Refilling buffer:  60%|██████    | 6/10 [00:01<00:00,  6.68it/s]


Refilling buffer:  80%|████████  | 8/10 [00:01<00:00,  7.89it/s]


Refilling buffer: 100%|██████████| 10/10 [00:01<00:00,  8.66it/s]


                                                      

InterruptedException: 

In [ ]:
# 2) Prepare SAE dict for upload
#    use the hook_name (e.g. "blocks.4.hook_mlp_out") as the folder key
sae_dict = {
    cfg.hook_name: sparse_autoencoder
}

# 3) Upload to HuggingFace
upload_saes_to_huggingface(
    saes=sae_dict,
    hf_repo_id="your-username/sae-pythia-410m-deduped",  # your target repo
    private=False                                       # set True if you want a private repo
)

print(f"✅ Uploaded SAE to https://huggingface.co/{'your-username/sae-pythia-410m-deduped'}")

Notes